In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd

from IPython.display import Image

In [3]:
tf.random.set_seed(1)
np.random.seed(1)

In [4]:
#Image(filename='images/02.png', width=700)

In [5]:
dataset_path = tf.keras.utils.get_file("auto-mpg.data", 
                                       ("http://archive.ics.uci.edu/ml/machine-learning-databases"
                                        "/auto-mpg/auto-mpg.data"))

column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower',
                'Weight', 'Acceleration', 'ModelYear', 'Origin']

df = pd.read_csv(dataset_path, names=column_names,
                 na_values = "?", comment='\t',
                 sep=" ", skipinitialspace=True)

df.tail()

30286/30286 [==============================] - 0s 6us/step


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [6]:
print(df.isna().sum())

df = df.dropna()
df = df.reset_index(drop=True)
df.tail()

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
ModelYear       0
Origin          0
dtype: int64


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
387,27.0,4,140.0,86.0,2790.0,15.6,82,1
388,44.0,4,97.0,52.0,2130.0,24.6,82,2
389,32.0,4,135.0,84.0,2295.0,11.6,82,1
390,28.0,4,120.0,79.0,2625.0,18.6,82,1
391,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [7]:
import sklearn
import sklearn.model_selection


df_train, df_test = sklearn.model_selection.train_test_split(df, train_size=0.8)
train_stats = df_train.describe().transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
MPG,313.0,23.404153,7.666909,9.0,17.5,23.0,29.0,46.6
Cylinders,313.0,5.402556,1.701506,3.0,4.0,4.0,8.0,8.0
Displacement,313.0,189.512780,102.675646,68.0,104.0,140.0,260.0,455.0
Horsepower,313.0,102.929712,37.919046,46.0,75.0,92.0,120.0,230.0
Weight,313.0,2961.198083,848.602146,1613.0,2219.0,2755.0,3574.0,5140.0
Acceleration,313.0,15.704473,2.725399,8.5,14.0,15.5,17.3,24.8
ModelYear,313.0,75.929712,3.675305,70.0,73.0,76.0,79.0,82.0
Origin,313.0,1.591054,0.807923,1.0,1.0,1.0,2.0,3.0


In [8]:
numeric_column_names = ['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration']

df_train_norm, df_test_norm = df_train.copy(), df_test.copy()

for col_name in numeric_column_names:
    mean = train_stats.loc[col_name, 'mean']
    std  = train_stats.loc[col_name, 'std']
    df_train_norm.loc[:, col_name] = (df_train_norm.loc[:, col_name] - mean)/std
    df_test_norm.loc[:, col_name] = (df_test_norm.loc[:, col_name] - mean)/std
    
df_train_norm.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
203,28.0,-0.824303,-0.901020,-0.736562,-0.950031,0.255202,76,3
255,19.4,0.351127,0.413800,-0.340982,0.293190,0.548737,78,1
72,13.0,1.526556,1.144256,0.713897,1.339617,-0.625403,72,1
235,30.5,-0.824303,-0.891280,-1.053025,-1.072585,0.475353,77,1
37,14.0,1.526556,1.563051,1.636916,1.470420,-1.359240,71,1


In [9]:
numeric_features = []

for col_name in numeric_column_names:
    numeric_features.append(tf.feature_column.numeric_column(key=col_name))
    
numeric_features

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[NumericColumn(key='Cylinders', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Displacement', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Horsepower', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Weight', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Acceleration', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [10]:
feature_year = tf.feature_column.numeric_column(key="ModelYear")

bucketized_features = []

bucketized_features.append(tf.feature_column.bucketized_column(
    source_column=feature_year,
    boundaries=[73, 76, 79]))

print(bucketized_features)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
[BucketizedColumn(source_column=NumericColumn(key='ModelYear', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(73, 76, 79))]


In [11]:
feature_origin = tf.feature_column.categorical_column_with_vocabulary_list(
    key='Origin',
    vocabulary_list=[1, 2, 3])

categorical_indicator_features = []
categorical_indicator_features.append(tf.feature_column.indicator_column(feature_origin))

print(categorical_indicator_features)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Origin', vocabulary_list=(1, 2, 3), dtype=tf.int32, default_value=-1, num_oov_buckets=0))]


In [12]:
def train_input_fn(df_train, batch_size=8):
    df = df_train.copy()
    train_x, train_y = df, df.pop('MPG')
    dataset = tf.data.Dataset.from_tensor_slices((dict(train_x), train_y))

    # shuffle, repeat, and batch the examples
    return dataset.shuffle(1000).repeat().batch(batch_size)

## inspection
ds = train_input_fn(df_train_norm)
batch = next(iter(ds))
print('Keys:', batch[0].keys())
print('Batch Model Years:', batch[0]['ModelYear'])

Keys: dict_keys(['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'ModelYear', 'Origin'])
Batch Model Years: tf.Tensor([82 78 76 72 78 73 70 78], shape=(8,), dtype=int64)


In [13]:
all_feature_columns = (numeric_features + 
                       bucketized_features + 
                       categorical_indicator_features)

print(all_feature_columns)

[NumericColumn(key='Cylinders', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Displacement', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Horsepower', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Weight', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Acceleration', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), BucketizedColumn(source_column=NumericColumn(key='ModelYear', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(73, 76, 79)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Origin', vocabulary_list=(1, 2, 3), dtype=tf.int32, default_value=-1, num_oov_buckets=0))]


In [14]:
regressor = tf.estimator.DNNRegressor(
    feature_columns=all_feature_columns,
    hidden_units=[32, 10],
    model_dir='models\\autompg-dnnregressor\\')

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models\\autompg-dnnregressor\\', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_

In [15]:
EPOCHS = 1000
BATCH_SIZE = 8
total_steps = EPOCHS * int(np.ceil(len(df_train) / BATCH_SIZE))
print('Training Steps:', total_steps)

regressor.train(
    input_fn=lambda:train_input_fn(df_train_norm, batch_size=BATCH_SIZE),
    steps=total_steps)

Training Steps: 40000
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:

INFO:tensorflow:loss = 523.63184, step = 3700 (0.053 sec)
INFO:tensorflow:global_step/sec: 1966.04
INFO:tensorflow:loss = 395.42133, step = 3800 (0.050 sec)
INFO:tensorflow:global_step/sec: 1966.06
INFO:tensorflow:loss = 434.71265, step = 3900 (0.051 sec)
INFO:tensorflow:global_step/sec: 2046.2
INFO:tensorflow:loss = 433.00592, step = 4000 (0.049 sec)
INFO:tensorflow:global_step/sec: 1966.08
INFO:tensorflow:loss = 555.7258, step = 4100 (0.051 sec)
INFO:tensorflow:global_step/sec: 1699.5
INFO:tensorflow:loss = 430.87704, step = 4200 (0.058 sec)
INFO:tensorflow:global_step/sec: 1822.27
INFO:tensorflow:loss = 255.30963, step = 4300 (0.057 sec)
INFO:tensorflow:global_step/sec: 1759.8
INFO:tensorflow:loss = 324.1206, step = 4400 (0.056 sec)
INFO:tensorflow:global_step/sec: 2046.19
INFO:tensorflow:loss = 533.502, step = 4500 (0.048 sec)
INFO:tensorflow:global_step/sec: 2005.38
INFO:tensorflow:loss = 499.69388, step = 4600 (0.051 sec)
INFO:tensorflow:global_step/sec: 2005.36
INFO:tensorflow:l

INFO:tensorflow:loss = 150.88309, step = 12000 (0.055 sec)
INFO:tensorflow:global_step/sec: 2365.68
INFO:tensorflow:loss = 134.913, step = 12100 (0.043 sec)
INFO:tensorflow:global_step/sec: 1591.53
INFO:tensorflow:loss = 124.31195, step = 12200 (0.062 sec)
INFO:tensorflow:global_step/sec: 1891.87
INFO:tensorflow:loss = 144.48666, step = 12300 (0.053 sec)
INFO:tensorflow:global_step/sec: 1759.08
INFO:tensorflow:loss = 135.53232, step = 12400 (0.057 sec)
INFO:tensorflow:global_step/sec: 1856.81
INFO:tensorflow:loss = 268.16122, step = 12500 (0.054 sec)
INFO:tensorflow:global_step/sec: 1891.81
INFO:tensorflow:loss = 75.51152, step = 12600 (0.053 sec)
INFO:tensorflow:global_step/sec: 1790.49
INFO:tensorflow:loss = 166.94037, step = 12700 (0.056 sec)
INFO:tensorflow:global_step/sec: 1790.48
INFO:tensorflow:loss = 170.14723, step = 12800 (0.055 sec)
INFO:tensorflow:global_step/sec: 1928.25
INFO:tensorflow:loss = 221.7654, step = 12900 (0.053 sec)
INFO:tensorflow:global_step/sec: 1891.83
INFO

INFO:tensorflow:global_step/sec: 1856.82
INFO:tensorflow:loss = 43.315533, step = 20300 (0.054 sec)
INFO:tensorflow:global_step/sec: 1966.03
INFO:tensorflow:loss = 20.881638, step = 20400 (0.050 sec)
INFO:tensorflow:global_step/sec: 2005.33
INFO:tensorflow:loss = 33.815044, step = 20500 (0.051 sec)
INFO:tensorflow:global_step/sec: 2046.34
INFO:tensorflow:loss = 23.123558, step = 20600 (0.049 sec)
INFO:tensorflow:global_step/sec: 2046.26
INFO:tensorflow:loss = 34.131393, step = 20700 (0.049 sec)
INFO:tensorflow:global_step/sec: 1790.47
INFO:tensorflow:loss = 21.784689, step = 20800 (0.056 sec)
INFO:tensorflow:global_step/sec: 1670.97
INFO:tensorflow:loss = 29.059, step = 20900 (0.060 sec)
INFO:tensorflow:global_step/sec: 1928.37
INFO:tensorflow:loss = 7.8652816, step = 21000 (0.051 sec)
INFO:tensorflow:global_step/sec: 1823.09
INFO:tensorflow:loss = 10.321878, step = 21100 (0.055 sec)
INFO:tensorflow:global_step/sec: 1790.52
INFO:tensorflow:loss = 59.088814, step = 21200 (0.057 sec)
INF

INFO:tensorflow:loss = 23.384895, step = 28500 (0.057 sec)
INFO:tensorflow:global_step/sec: 2046.28
INFO:tensorflow:loss = 31.130472, step = 28600 (0.049 sec)
INFO:tensorflow:global_step/sec: 2005.33
INFO:tensorflow:loss = 6.5732145, step = 28700 (0.050 sec)
INFO:tensorflow:global_step/sec: 2088.86
INFO:tensorflow:loss = 10.567648, step = 28800 (0.048 sec)
INFO:tensorflow:global_step/sec: 2088.91
INFO:tensorflow:loss = 42.739845, step = 28900 (0.048 sec)
INFO:tensorflow:global_step/sec: 1891.87
INFO:tensorflow:loss = 10.148725, step = 29000 (0.053 sec)
INFO:tensorflow:global_step/sec: 2046.26
INFO:tensorflow:loss = 6.210287, step = 29100 (0.049 sec)
INFO:tensorflow:global_step/sec: 1928.21
INFO:tensorflow:loss = 32.67692, step = 29200 (0.052 sec)
INFO:tensorflow:global_step/sec: 1790.5
INFO:tensorflow:loss = 13.740294, step = 29300 (0.056 sec)
INFO:tensorflow:global_step/sec: 1643.7
INFO:tensorflow:loss = 14.542434, step = 29400 (0.061 sec)
INFO:tensorflow:global_step/sec: 2045.07
INFO

INFO:tensorflow:global_step/sec: 2179.78
INFO:tensorflow:loss = 23.79562, step = 36800 (0.046 sec)
INFO:tensorflow:global_step/sec: 2179.65
INFO:tensorflow:loss = 14.197407, step = 36900 (0.046 sec)
INFO:tensorflow:global_step/sec: 2179.81
INFO:tensorflow:loss = 15.19112, step = 37000 (0.045 sec)
INFO:tensorflow:global_step/sec: 2179.73
INFO:tensorflow:loss = 2.003418, step = 37100 (0.047 sec)
INFO:tensorflow:global_step/sec: 2228.18
INFO:tensorflow:loss = 16.520327, step = 37200 (0.045 sec)
INFO:tensorflow:global_step/sec: 2179.64
INFO:tensorflow:loss = 7.276082, step = 37300 (0.046 sec)
INFO:tensorflow:global_step/sec: 2179.8
INFO:tensorflow:loss = 10.472056, step = 37400 (0.047 sec)
INFO:tensorflow:global_step/sec: 2045.21
INFO:tensorflow:loss = 8.011268, step = 37500 (0.048 sec)
INFO:tensorflow:global_step/sec: 2134.45
INFO:tensorflow:loss = 16.085062, step = 37600 (0.046 sec)
INFO:tensorflow:global_step/sec: 2278.84
INFO:tensorflow:loss = 7.222141, step = 37700 (0.045 sec)
INFO:te

In [16]:
reloaded_regressor = tf.estimator.DNNRegressor(
    feature_columns=all_feature_columns,
    hidden_units=[32, 10],
    warm_start_from='models\\autompg-dnnregressor\\',
    model_dir='models\\autompg-dnnregressor\\')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models\\autompg-dnnregressor\\', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use tf.keras instead.


In [17]:
def eval_input_fn(df_test, batch_size=8):
    df = df_test.copy()
    test_x, test_y = df, df.pop('MPG')
    dataset = tf.data.Dataset.from_tensor_slices((dict(test_x), test_y))

    return dataset.batch(batch_size)

eval_results = reloaded_regressor.evaluate(
    input_fn=lambda:eval_input_fn(df_test_norm, batch_size=8))

for key in eval_results:
    print('{:15s} {}'.format(key, eval_results[key]))
    
print('Average-Loss {:.4f}'.format(eval_results['average_loss']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-05-23T10:37:19
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models\autompg-dnnregressor\model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.92224s
INFO:tensorflow:Finished evaluation at 2023-05-23-10:37:20
INFO:tensorflow:Saving dict for global step 40000: average_loss = 17.161558, global_step = 40000, label/mean = 23.611393, loss = 17.058157, prediction/mean = 21.800117
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 40000: models\autompg-dnnregressor\model.ckpt-40000
average_loss    17.161558151245117
label/mean      23.611392974853516
loss            17.058156967163086
prediction/mean 21.80011749267578
global_step     40000
Average-Loss 17.1616


In [18]:
pred_res = regressor.predict(input_fn=lambda: eval_input_fn(df_test_norm, batch_size=8))

print(next(iter(pred_res)))

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models\autompg-dnnregressor\model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([22.622843], dtype=float32)}


In [20]:
boosted_tree = tf.estimator.BoostedTreesRegressor(
    feature_columns=all_feature_columns,
    n_batches_per_layer=20,
    n_trees=200)

boosted_tree.train(
    input_fn=lambda:train_input_fn(df_train_norm, batch_size=BATCH_SIZE))

eval_results = boosted_tree.evaluate(
    input_fn=lambda:eval_input_fn(df_test_norm, batch_size=8))

print(eval_results)

print('Average-Loss {:.4f}'.format(eval_results['average_loss']))

AttributeError: module 'tensorflow_estimator.python.estimator.api._v2.estimator' has no attribute 'BoostedTreesRegressor'

In [21]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 20
steps_per_epoch = np.ceil(60000 / BATCH_SIZE)

In [22]:
def preprocess(item):
    image = item['image']
    label = item['label']
    image = tf.image.convert_image_dtype(
        image, tf.float32)
    image = tf.reshape(image, (-1,))

    return {'image-pixels':image}, label[..., tf.newaxis]

#Step 1: Defining the input functions (one for training and one for evaluation)
## Step 1: Define the input function for training
def train_input_fn():
    datasets = tfds.load(name='mnist')
    mnist_train = datasets['train']

    dataset = mnist_train.map(preprocess)
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset.repeat()

## define input-function for evaluation:
def eval_input_fn():
    datasets = tfds.load(name='mnist')
    mnist_test = datasets['test']
    dataset = mnist_test.map(preprocess).batch(BATCH_SIZE)
    return dataset

In [23]:
## Step 2: feature column
image_feature_column = tf.feature_column.numeric_column(
    key='image-pixels', shape=(28*28))

In [25]:
## Step 3: instantiate the estimator
dnn_classifier = tf.estimator.DNNClassifier(
    feature_columns=[image_feature_column],
    hidden_units=[32, 16],
    n_classes=10,
    model_dir='models\\mnist-dnn\\')


## Step 4: train
dnn_classifier.train(
    input_fn=train_input_fn,
    steps=NUM_EPOCHS * steps_per_epoch)

RuntimeError: Graph is finalized and cannot be modified.

In [26]:
Image(filename='images/15_01.png', width=700)

FileNotFoundError: [Errno 2] No such file or directory: 'images/15_01.png'

In [27]:
Image(filename='images/15_03.png', width=700)

FileNotFoundError: [Errno 2] No such file or directory: 'images/15_03.png'

In [28]:
Image(filename='images/15_04.png', width=700)

FileNotFoundError: [Errno 2] No such file or directory: 'images/15_04.png'

In [29]:
import tensorflow as tf
import numpy as np

print('TensorFlow version:', tf.__version__)
print('NumPy version: ', np.__version__)

TensorFlow version: 2.12.0
NumPy version:  1.22.4


In [30]:
def conv1d(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate(
            [zero_pad, x_padded, zero_pad])
    res = []
    for i in range(0, int((len(x_padded) - len(w_rot)) / s) + 1, s):
        res.append(np.sum(
            x_padded[i:i+w_rot.shape[0]] * w_rot))
    return np.array(res)


## Testing:
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]

print('Conv1d Implementation:',
      conv1d(x, w, p=2, s=1))

print('Numpy Results:',
      np.convolve(x, w, mode='same')) 

Conv1d Implementation: [ 5. 14. 16. 26. 24. 34. 19. 22.]
Numpy Results: [ 5 14 16 26 24 34 19 22]
